# AI Music Generation

### Preprocessing MIDI files

##### Import required packages

In [0]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle

##### Read and play a sample MIDI file

In [0]:
sample_file = converter.parse("songs/EyesOnMePiano.mid")

In [6]:
print(type(sample_file))

<class 'music21.stream.Score'>


In [7]:
# Play the song
sample_file.show("midi")

In [8]:
# Output song as text
sample_file.show("text")

{0.0} <music21.stream.Part 0x7f04883097b8>
    {0.0} <music21.instrument.Piano Piano>
    {0.0} <music21.tempo.MetronomeMark Quarter=95.0>
    {0.0} <music21.key.Key of D major>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.stream.Voice 0x7f04881112b0>
        {0.0} <music21.note.Note A>
        {0.0} <music21.note.Note A>
        {0.6667} <music21.note.Note G>
        {1.5} <music21.note.Note D>
        {2.0} <music21.note.Note A>
        {2.5} <music21.note.Note G>
        {3.0} <music21.note.Note D>
        {3.5} <music21.note.Note A>
        {4.0} <music21.note.Note A>
        {6.0} <music21.note.Note A>
        {8.0} <music21.note.Note F#>
        {10.0} <music21.chord.Chord F#4 C#5>
        {11.0} <music21.note.Note A>
        {11.6667} <music21.note.Note D>
        {13.3333} <music21.note.Note D>
        {13.6667} <music21.note.Note C#>
        {14.0} <music21.note.Note D>
        {15.0} <music21.note.Note D>
        {16.0} <music21.note.Note G>
        {16.6667

In [9]:
# The above test output is like an array of arrays
# So flatten it
sample_file_flattened = sample_file.flat.notes
print(len(sample_file_flattened))

1421


In [10]:
for element in sample_file_flattened:
    # Print the element and its offset (time from starting when it is played)
    print(element, element.offset)

<music21.note.Note A> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 0.25
<music21.note.Note G> 2/3
<music21.note.Note G> 2/3
<music21.note.Note F#> 1.0
<music21.note.Note F#> 1.25
<music21.note.Note D> 1.5
<music21.note.Note D> 1.5
<music21.note.Note C#> 1.75
<music21.note.Note C#> 1.75
<music21.note.Note A> 2.0
<music21.note.Note A> 2.25
<music21.note.Note G> 2.5
<music21.note.Note G> 2.5
<music21.note.Note F#> 2.75
<music21.note.Note F#> 2.75
<music21.note.Note D> 3.0
<music21.note.Note D> 3.25
<music21.note.Note A> 3.5
<music21.note.Note A> 3.5
<music21.note.Note D> 3.75
<music21.note.Note D> 3.75
<music21.note.Note A> 4.0
<music21.note.Note D> 4.0
<music21.note.Note B> 4.0
<music21.note.Note G> 4.0
<music21.note.Note A> 4.25
<music21.note.Note D> 4.25
<music21.note.Note B> 4.25
<music21.note.Note G> 4.25
<music21.note.Note E> 5.0
<music21.note.Note C#> 5.0
<music21.note.Note E> 5.25
<music21.note.Note C#> 5.25
<music21.note.Note F#> 17/3
<music21.not

In [0]:
sample_notes = []

for element in sample_file_flattened:
    # If the element is a note, store its pitch as a string
    if isinstance(element, note.Note):
        sample_notes.append(str(element.pitch))

    # If the element is a chord, split it and join the node IDs together into a single sring separated by a '+'
    elif isinstance(element, chord.Chord):
        sample_notes.append("+".join(str(n) for n in element.normalOrder))

In [12]:
print(sample_notes)

['A5', 'A6', 'A5', 'A6', 'G6', 'G6', 'F#6', 'F#6', 'D6', 'D6', 'C#6', 'C#6', 'A5', 'A5', 'G5', 'G5', 'F#5', 'F#5', 'D5', 'D5', 'A4', 'A4', 'D5', 'D5', 'A4', 'D5', 'B3', 'G4', 'A4', 'D5', 'B3', 'G4', 'E5', 'C#4', 'E5', 'C#4', 'F#5', 'D4', 'F#5', 'D4', 'A4', 'E5', 'C#4', 'F#4', 'A4', 'E5', 'C#4', 'F#4', 'C#5', 'A3', 'C#5', 'A3', 'F#4', 'B4', 'G3', 'E4', 'F#4', 'B4', 'G3', 'E4', 'C#5', 'A3', 'C#5', 'A3', 'D5', 'B3', 'D5', 'B3', '1+6', '9+2', '1+6', '9+2', 'A3', 'A3', 'D4', 'D4', 'B3', 'E4', 'G2', 'B3', 'E4', 'G2', 'D3', 'D3', 'G3', 'G3', 'D4', 'D4', 'C#4', 'C#4', 'D4', 'D4', 'A4', '6+9', 'D4', 'D4', 'A4', '6+9', 'D5', 'D5', 'D4', 'G#4', '5+11', 'D4', 'G#4', '5+11', 'D5', 'D5', 'G4', 'E2', 'G4', 'E2', 'F#4', 'B2', 'F#4', 'B2', 'G4', 'E3', 'G4', 'E3', 'D5', 'G3', 'D5', 'G3', 'D4', 'B3', 'D4', 'B3', 'F#4', 'F#4', 'E4', 'E4', 'A2', 'A2', 'E3', 'E3', 'G3', 'G3', 'B3', 'B3', 'C#4', 'F4', '7+9', 'C#4', 'F4', '7+9', 'C#5', 'C#5', 'B4', 'B4', 'A4', 'D2', 'A4', 'D2', 'A2', 'A2', 'D3', 'D3', '6+9', 

In [0]:
print(len(sample_notes))

1421


### The above preprocessing was done just for one file, as an example.
### Now the same process is done for all the files in songs directory

In [15]:
notes = []

for file in glob.glob("songs/*.mid"):
    print("Parsing %s..." %file)
    parsed_file = converter.parse(file)
    flattened_file = parsed_file.flat.notes

    for element in flattened_file:
        # If the element is a note
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))

        # If the element is a chord
        elif isinstance(element, chord.Chord):
            notes.append("+".join(str(n) for n in element.normalOrder))

Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/Ff7-Jenova_Absolute.mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/FF8_Shuffle_or_boogie_pc.mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/ahead_on_our_way_piano.mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/DOS.mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/Suteki_Da_Ne_(Piano_Version).mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/FF3_Battle_(Piano).mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/balamb.mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/caitsith.mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/sera_.mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/Oppressed.mid...
Parsing /content/drive/My Drive/Personal/AI Music Generation/songs/ff7themep.mid...
Parsing /content/drive/M

In [16]:
print(len(notes))

60498


In [0]:
# Save these notes
with open ("notes", "wb") as filepath:
    pickle.dump (notes, filepath)